<a href="https://colab.research.google.com/github/lulujingyi/PreliminaryTest/blob/main/nonpre_XOR_stdp_cgp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
import sys
sys.path.insert(0,"/content/gdrive/My Drive/MasterThesis/hal-cgp-master")

Mounted at /content/gdrive


In [ ]:
import cgp
import functools
import warnings
import numpy as np

import math
from typing import Dict, Optional, Tuple, NamedTuple, Any, Sequence

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
# time constants (in ms)
TAU_V = 10.0  # voltage
V_DECAY = torch.exp(-1.0 / torch.tensor(TAU_V))
TAU_I = 2.0  # current
I_DECAY = torch.exp(-1.0 / torch.tensor(TAU_I))
TAU_O1 = 20.0  # neuron trace1. 5,10,15,20,30,40
TAU_O2= 40.0  # neuron trace2.  10,20,30,40,60,80
O_DECAY1 = torch.exp(-1.0 / torch.tensor(TAU_O1))
O_DECAY2 = torch.exp(-1.0 / torch.tensor(TAU_O2))
TAU_E = 100.0 # synapse trace (from paper)
E_DECAY = torch.exp(-1.0 / torch.tensor(TAU_E))
# thresh
THRESH = torch.tensor(1.0)

# inputs

N = 2    # neurons in spiking pop
M = 10   # sequences
T = 50   # length of single seq 500
F = 20    # spike freq  30

# learning
ETA = 0.001
EPOCHS = 300

In [ ]:
class SpikeFunction(torch.autograd.Function):
    """
    Spiking function with rectangular gradient.
    Source: https://www.frontiersin.org/articles/10.3389/fnins.2018.00331/full
    Implementation: https://github.com/combra-lab/pop-spiking-deep-rl/blob/main/popsan_drl/popsan_td3/popsan.py
    """

    @staticmethod
    def forward(ctx: Any, v: torch.Tensor) -> torch.Tensor:
        ctx.save_for_backward(v)  # save voltage - thresh for backwards pass
        return v.gt(0.0).float()

    @staticmethod
    def backward(ctx: Any, grad_output: torch.Tensor) -> Tuple[torch.Tensor, ...]:
        v, = ctx.saved_tensors
        grad_input = grad_output.clone()
        spike_pseudo_grad = (v.abs() < 0.5).float()  # 0.5 is the width of the rectangle
        return grad_input * spike_pseudo_grad, None  # ensure a tuple is returned

In [ ]:
class LIFState(NamedTuple):
    z: torch.Tensor
    v: torch.Tensor
    i: torch.Tensor
    o1: torch.Tensor
    o2: torch.Tensor

class LIF(nn.Module):
    """
    Leaky-integrate-and-fire neuron with learnable parameters.
    """

    def __init__(self, size: int):
        super().__init__()
        self.size = size
        # Initialize all parameters randomly as U(0, 1)
        self.i_decay = torch.ones(size)*I_DECAY #self.i_decay = nn.Parameter(torch.rand(size))
        self.v_decay = torch.ones(size)*V_DECAY
        self.o_decay1 = torch.ones(size)*O_DECAY1
        self.o_decay2 = torch.ones(size)*O_DECAY2
        self.thresh = torch.ones(size)*THRESH
        self.spike = SpikeFunction.apply  # spike function


    def forward(
        self,
        synapse: nn.Module,
        z: torch.Tensor,
        state: Optional[LIFState] = None,
    ) -> Tuple[torch.Tensor, LIFState]:
        # Previous state
        if state is None:
            state = LIFState(
                z=torch.zeros_like(synapse(z)),
                v=torch.zeros_like(synapse(z)),
                i=torch.zeros_like(synapse(z)),
                o1=torch.zeros_like(synapse(z)),
                o2=torch.zeros_like(synapse(z)),
            )
        # Update state
        i = state.i * self.i_decay + synapse(z)
        v = state.v * self.v_decay * (1.0 - state.z) + i
        z = self.spike(v - self.thresh)
        o1 = state.o1 * self.o_decay1 - z
        o2 = state.o2 * self.o_decay2 - z

        return z, LIFState(z, v, i, o1, o2)

In [ ]:
class SpikingMLP(nn.Module):
    """
    Spiking network with LIF neuron model.
    """

    def __init__(self, sizes: Sequence[int]):
        super().__init__()
        self.sizes = sizes
        self.spike = SpikeFunction.apply

        # Define layers
        self.synapses = nn.ModuleList()
        self.neurons = nn.ModuleList()
        self.states = []
        # Loop over current (accessible with 'size') and next (accessible with 'sizes[i]') element
        for i, size in enumerate(sizes[:-1], start=1):
            # Parameters of synapses and neurons are randomly initialized
            self.synapses.append(nn.Linear(size, sizes[i], bias=False))
            self.neurons.append(LIF(sizes[i]))
            self.states.append(None)
           

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        for i, (neuron, synapse) in enumerate(zip(self.neurons, self.synapses)):
            z, self.states[i] = neuron(synapse, z, self.states[i])
        return z

    def reset(self):
        """
        Resetting states when you're done is very important!
        """
        for i, _ in enumerate(self.states):
            self.states[i] = None

In [ ]:
def encoding(inp):
    #height, width = x.shape
    inp = inp.unsqueeze(-1)
    rate = 0.2
    return (torch.rand((len(inp),T))<(inp*rate+0.1)).float()

In [ ]:
def generate_input():
  data = torch.zeros([M,T,2])
  x = torch.randint(2, (M, 2)).float()
  targets = (x.sum(-1) == 1).float()
  for i in range(M):
    data[i] = encoding(x[i]).reshape(T,2)
  return targets,data
#targets,data = generate_input()
#SIZE: 10,50,2 (m,t,2)

In [ ]:
def learn(f,i,t, ETA, E11, E12, E21, E22, E31, E32, z1, z2, z3, reward, W):
    w1 = torch.zeros_like(W[0])
    for p in range(w1.size()[1]):
        for q in range(w1.size()[0]):
            w1[q][p] = torch.tensor(f([E11[i,t,p],E12[i,t,p],E21[q],E22[q],z1[i,t,p],z2[q], reward]))*ETA          

    w2 = torch.zeros_like(W[1])
    for p in range(w2.size()[1]):
        for q in range(w2.size()[0]):
            w2[q][p] = torch.tensor(f([E21[p], E22[p], E31[q], E32[q], z2[p], z3[q], reward]))*ETA
    return w1,w2


def get_reward(z: torch.Tensor, label: int) -> Tuple[int, int]:
    # return reward and correct for counter
    #If the correct output was 1,the network received a reward r = 1 for each output spike emitted and 0 otherwise
    #If the correct output was 0, the network received a negative reward (punishment) r = −1 for each output spike and 0 otherwise.
    if z.item() == 1:
        if label == 1:
            return 1 
        else:
            return -1  
    elif z.item() == 0:
        return 0  
    else:
        raise ValueError("y negative")

In [ ]:
sizes = [2,5,1]
snn = SpikingMLP(sizes)
snn(torch.ones(2))
print(snn.states[0][0])

tensor([0., 0., 0., 0., 0.], grad_fn=<SpikeFunctionBackward>)


In [ ]:
  targets,data = generate_input()
  traces1 = []
  traces2 = []
  # ugly
  for i in range(data.shape[1]):
      if i == 0:
          traces1.append(data[:, i, :])
          traces2.append(data[:, i, :])
      else:
          trace1 = traces1[-1] * O_DECAY1 + data[:, i, :] 
          traces1.append(trace1)
          trace2 = traces2[-1] * O_DECAY2 + data[:, i, :] 
          traces2.append(trace2)
  traces1 = torch.stack(traces1, dim=1)
  traces2 = torch.stack(traces2, dim=1)

In [ ]:
print(traces1.size())
print(data.size())

torch.Size([10, 50, 2])
torch.Size([10, 50, 2])


In [ ]:
def inner_objective(f,seed):  
  torch.manual_seed(seed)
  targets,data = generate_input()
  traces1 = []
  traces2 = []
  # ugly
  for i in range(data.shape[1]):
      if i == 0:
          traces1.append(data[:, i, :])
          traces2.append(data[:, i, :])
      else:
          trace1 = traces1[-1] * O_DECAY1 + data[:, i, :] 
          traces1.append(trace1)
          trace2 = traces2[-1] * O_DECAY2 + data[:, i, :] 
          traces2.append(trace2)
  traces1 = torch.stack(traces1, dim=1)
  traces2 = torch.stack(traces2, dim=1)
  param = {}
  # weights from N(0, 1000) like in paper, sparsity of 0.2 (20% of connections zero)
  param["W"] = torch.randn(1, N) * 10 * (torch.rand(1, N) < 0.8).float()
  #print(param)
  acc = []
  fit = []
  ne = 140
  nf = 10
  sizes = [2,5,1]
  snn = SpikingMLP(sizes)
  for e in range(ne):
      correct = 0
      ytab = []
      for i in range(M):
          s = None
          y = 0
          E = None  # trace from paper
          for t in range(T): #T
              #if E is None:
                  #E = torch.zeros_like(param["W"])
              #z, s = forward(data[i, t], s, param)
              z = snn(data[i,t])
              y += z
              #xi = traces[i, t] * z + s[3] * data[i, t] 
              #E = E * E_DECAY + xi  
              reward = get_reward(z, targets[i])
              w1, w2 = learn(f,i,t, ETA, traces1, traces2, snn.states[0][3], snn.states[0][4], snn.states[1][3], snn.states[1][4], data, snn.states[0][0], snn.states[1][0], reward, list(snn.parameters()))
              #dw = torch.tensor(f([ETA,E,reward])) #ETA*E*reward
              #print(dw)
              for idx,w in enumerate(snn.parameters()):
                  if idx == 0:
                      w.data = w1 + w.data
                  else:
                      w.data = w2 + w.data
          ytab.append(y.item())
          
      predictions = torch.Tensor([1 if n > sum(ytab)/M else 0 for n in ytab])
      correct = int(M-sum(abs(predictions-targets)).item())
      acc.append(correct/M)
      #print(f"epoch {e}: {correct}/{M}")
      if e >= (ne - nf):
        fit.append(correct)
  fitness = sum(fit)/nf     
  print('finish training')   
  return fitness

In [ ]:
def objective(ind,seed):
    if not ind.fitness_is_None():
      return ind
    f = ind.to_func()
    try:
      with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore", message="divide by zero encountered in double_scalars"
            )
            warnings.filterwarnings(
                "ignore", message="invalid value encountered in double_scalars"
            )
            accuracy = inner_objective(f,seed)
      ind.fitness = accuracy
    except ZeroDivisionError:
      ind.fitness = -np.inf
    return ind

In [ ]:
def evolve(seed):

    population_params = {"n_parents": 2, "seed": seed}

    genome_params = {
        "n_inputs": 7,
        "n_outputs": 1,
        "n_columns": 12,
        "n_rows": 2,
        "levels_back": 5,
        "primitives": (
            cgp.Add,
            cgp.Sub,
            cgp.Mul,
            cgp.Div,
            cgp.ConstantFloat,
        ),
    }

    ea_params = {"n_offsprings": 4, "tournament_size": 1, "mutation_rate": 0.03, "n_processes": 3}

    evolve_params = {"max_generations": 100, "min_fitness": 30.0}

    pop = cgp.Population(**population_params, genome_params=genome_params)

    ea = cgp.ea.MuPlusLambda(**ea_params)

    history = {}
    history["expr_champion"] = []
    history["fitness_champion"] = []

    def recording_callback(pop):
        history["expr_champion"].append(pop.champion.to_sympy())
        history["fitness_champion"].append(pop.champion.fitness)

    obj = functools.partial(objective, seed=population_params["seed"])

    cgp.evolve(pop, obj, ea, **evolve_params, print_progress=True, callback=recording_callback)

    return history, pop.champion

In [ ]:
seed = 0
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[2/100] max fitness: 5.0finish training
[3/100] max fitness: 5.0finish training
finish training
[5/100] max fitness: 5.0finish training
finish training
finish training
[6/100] max fitness: 5.0finish training
finish training
[7/100] max fitness: 5.0finish training
[9/100] max fitness: 5.0finish training
finish training
finish training
[10/100] max fitness: 5.0finish training
finish training
[11/100] max fitness: 5.0finish training
[12/100] max fitness: 5.0finish training
finish training
finish training
[14/100] max fitness: 5.0finish training
finish training
[15/100] max fitness: 5.0finish training
[49/100] max fitness: 5.0finish training
[53/100] max fitness: 5.0finish training
[55/100] max fitness: 5.0finish training
[57/100] max fitness: 5.0finish training
[70/100] max fitness: 5.0finish training
[76/100] max fitness: 5.0finish training
[80/100] max fitness: 5.0finish training
[81/100] max fitness: 5.0finish training
[85/100] max fitnes

In [ ]:
seed = 1
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[2/100] max fitness: 5.0finish training
finish training
[4/100] max fitness: 5.0finish training
[5/100] max fitness: 5.0finish training
[6/100] max fitness: 5.0finish training
finish training
[7/100] max fitness: 5.0finish training
[8/100] max fitness: 5.0finish training
finish training
[9/100] max fitness: 5.0finish training
[12/100] max fitness: 5.0finish training
[13/100] max fitness: 5.0finish training
[14/100] max fitness: 5.0finish training
finish training
[16/100] max fitness: 5.0finish training
[18/100] max fitness: 5.0finish training
finish training
[22/100] max fitness: 5.0finish training
[23/100] max fitness: 5.0finish training
[24/100] max fitness: 5.0finish training
[25/100] max fitness: 5.0finish training
[26/100] max fitness: 5.0finish training
[27/100] max fitness: 5.0finish training
[29/100] max fitness: 5.0finish training
[30/100] max fitness: 5.0finish training
[32/100] max fitness: 5.0finish training
finish training
[3

In [ ]:
seed = 2
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[2/100] max fitness: 4.0finish training
finish training
[4/100] max fitness: 4.0finish training
[6/100] max fitness: 4.3finish training
[7/100] max fitness: 4.3finish training
[10/100] max fitness: 4.3finish training
[11/100] max fitness: 4.3finish training
[12/100] max fitness: 4.3finish training
[15/100] max fitness: 4.3finish training
[23/100] max fitness: 4.3finish training
[25/100] max fitness: 4.3finish training
[30/100] max fitness: 4.3finish training
[32/100] max fitness: 4.3finish training
[34/100] max fitness: 4.3finish training
[35/100] max fitness: 4.3finish training
[37/100] max fitness: 4.3finish training
[39/100] max fitness: 4.3finish training
[40/100] max fitness: 4.3finish training
[44/100] max fitness: 4.3finish training
finish training
[48/100] max fitness: 4.3finish training
[50/100] max fitness: 4.3finish training
[52/100] max fitness: 4.3finish training
finish training
[54/100] max fitness: 4.3finish training
finish

In [ ]:
seed = 3
history, champion = evolve(seed)
print(history)

finish training
finish training
[2/100] max fitness: 6.0finish training
[3/100] max fitness: 6.0finish training
[5/100] max fitness: 6.0finish training
[14/100] max fitness: 6.0finish training
[16/100] max fitness: 6.0finish training
[18/100] max fitness: 6.0finish training
[20/100] max fitness: 6.0finish training
[22/100] max fitness: 6.0finish training
[25/100] max fitness: 6.0finish training
[30/100] max fitness: 6.0finish training
[31/100] max fitness: 6.0finish training
[32/100] max fitness: 6.0finish training
[35/100] max fitness: 6.0finish training
[36/100] max fitness: 6.0finish training
[37/100] max fitness: 6.0finish training
[38/100] max fitness: 6.0finish training
[39/100] max fitness: 6.0finish training
[43/100] max fitness: 6.0finish training
[44/100] max fitness: 6.0finish training
finish training
finish training
[45/100] max fitness: 6.0finish training
[47/100] max fitness: 6.0finish training
[48/100] max fitness: 6.0finish training
[49/100] max fitness: 6.0finish train

In [ ]:
seed = 0
history, champion = evolve(seed)
print(history)

finish training
finish training
[10/10] max fitness: 5.0
{'expr_champion': [[-x_0 + x_3 + 1.0], [x_0], [x_0], [x_0], [x_0], [x_0], [x_0], [x_0], [x_0], [x_0]], 'fitness_champion': [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]}


In [ ]:
seed = 1
history, champion = evolve(seed)
print(history)

finish training
finish training
[6/10] max fitness: 5.0finish training
[8/10] max fitness: 5.0finish training
[10/10] max fitness: 5.0
{'expr_champion': [[1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000]], 'fitness_champion': [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]}


In [ ]:
seed = 2
history, champion = evolve(seed)
print(history)

finish training
finish training
[3/10] max fitness: 4.0finish training
[5/10] max fitness: 4.0finish training
[6/10] max fitness: 4.0finish training
[7/10] max fitness: 4.0finish training
[8/10] max fitness: 4.0finish training
finish training
[9/10] max fitness: 4.0finish training
[10/10] max fitness: 4.0
{'expr_champion': [[1.0*x_2], [1.0*x_2], [1.0*x_2], [1.0*x_1], [1.0*x_2], [1.0*x_2], [1.0*x_2**2*x_4], [1.0*x_2**2*x_4], [1.0 - x_2], [1.0*x_2*(x_2 - x_4)]], 'fitness_champion': [4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]}


In [ ]:
seed = 3
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[5/10] max fitness: 6.0finish training
finish training
[8/10] max fitness: 6.0finish training
[10/10] max fitness: 6.0
{'expr_champion': [[2*x_0*x_1 + x_1 + x_4], [0], [0], [0], [0], [1.0 - 1.0*x_0], [0], [0], [0], [0]], 'fitness_champion': [6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0]}


In [ ]:
seed = 4
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[2/10] max fitness: 6.0finish training
[5/10] max fitness: 6.0finish training
[7/10] max fitness: 6.0finish training
[8/10] max fitness: 8.0finish training
[9/10] max fitness: 8.0finish training
finish training
[10/10] max fitness: 8.0
{'expr_champion': [[x_3 + 1/(x_1*x_4)], [x_2/(x_3*x_4) + x_3], [x_3 + 1/(x_0*x_1)], [x_3 + 1/(x_0*x_1)], [x_3 + 1/(x_0*x_1)], [x_3 + 1.0*x_4], [x_3 + 1/(x_0*x_1)], [1.0*x_0*x_1 + x_3], [1.0*x_0*x_1 + x_3], [1.0*x_0*x_1 + x_3]], 'fitness_champion': [6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 8.0, 8.0, 8.0]}


===========================================

In [ ]:
seed = 1
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
finish training
[3/10] max fitness: 19.0finish training
[4/10] max fitness: 19.0finish training
[5/10] max fitness: 19.0finish training
finish training
[6/10] max fitness: 19.0finish training
[8/10] max fitness: 19.0finish training
finish training
[9/10] max fitness: 19.0finish training
[10/10] max fitness: 19.0
{'expr_champion': [[1.0*x_0 + 0.5], [1.0*x_0*x_1 + 1.0], [1.0*x_0*x_1 + 1.0], [1.0*x_0*x_1 + 1.0], [1.0*x_0*x_1 + 1.0], [1.0*x_0*x_1 + 1.0], [1.0*x_0*x_1 + 1.0], [1.0*x_0*x_1 + 1.0], [1.0*x_0*x_1 + 1.0], [1.0*x_0*x_1 + 1.0]], 'fitness_champion': [14.6, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0]}


In [ ]:
seed = 2
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[2/10] max fitness: 13.0finish training
[4/10] max fitness: 13.0finish training
[5/10] max fitness: 13.0finish training
finish training
[6/10] max fitness: 13.0finish training
[8/10] max fitness: 13.0finish training
[10/10] max fitness: 13.0
{'expr_champion': [[(x_0 + x_1)*(-x_0 + 2*x_1**2 - x_1)/(2*x_1)], [-x_0*(x_0 + x_1) - x_0 - 2*x_1], [-x_0*x_1 - x_0 - 2*x_1], [-x_0*(x_0 + x_1) - x_0 - 2*x_1], [-x_0*(x_0 + x_1) - x_0 - 2*x_1], [(x_0 - x_1*(x_0*(x_0 + x_1) + 2*x_1) + x_1)/x_1], [(x_0 - x_1*(x_0*(x_0 + x_1) + 2*x_1) + x_1)/x_1], [(x_0 - x_1*(x_0*(x_0 + x_1) + 2*x_1) + x_1)/x_1], [(x_0 - x_1*(x_0*(x_0 + x_1) + 2*x_1) + x_1)/x_1], [(x_0 - x_1*(x_0*(x_0 + x_1) + 2*x_1) + x_1)/x_1]], 'fitness_champion': [13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0]}


In [ ]:
seed = 3
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[4/10] max fitness: 17.0finish training
[9/10] max fitness: 17.0finish training
[10/10] max fitness: 17.0
{'expr_champion': [[0], [x_1*(x_0 + 1)/(4*x_0)], [0], [0], [0], [0], [0], [0], [0], [0]], 'fitness_champion': [17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0]}


In [ ]:
seed = 4
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[4/10] max fitness: 22.9finish training
[6/10] max fitness: 28.0finish training
[9/10] max fitness: 28.0finish training
[10/10] max fitness: 28.0
{'expr_champion': [[x_1*(x_0 + 1)], [x_1*(x_0 + 1)], [x_1*(x_0 + 1)], [x_1*(x_0 + 1)], [x_0*x_1*(x_0 + 1)*(x_0 - x_1)*(2*x_0 - x_1)], [x_0*x_1*(x_0 + 1)*(x_0 - x_1)*(2*x_0 - x_1)], [x_0*x_1*(x_0 + 1)*(x_0 - x_1)*(2*x_0 - x_1)], [x_0*x_1*(x_0 + 1)*(x_0 - x_1)*(2*x_0 - x_1)], [x_0*x_1*(x_0 + 1)*(x_0 - x_1)*(2*x_0 - x_1)], [x_0*x_1*(x_0 + 1)*(x_0 - x_1)*(2*x_0 - x_1)]], 'fitness_champion': [22.9, 22.9, 22.9, 22.9, 28.0, 28.0, 28.0, 28.0, 28.0, 28.0]}


In [ ]:
seed = 5
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[2/10] max fitness: 17.0finish training
[3/10] max fitness: 17.0finish training
[4/10] max fitness: 17.0finish training
[6/10] max fitness: 17.0finish training
[9/10] max fitness: 17.0finish training
finish training
[10/10] max fitness: 17.0
{'expr_champion': [[1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000]], 'fitness_champion': [17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0]}


In [ ]:
seed = 6
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[3/10] max fitness: 29.0finish training
[4/10] max fitness: 29.0finish training
[10/10] max fitness: 29.0
{'expr_champion': [[x_1*(x_1 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)]], 'fitness_champion': [15.0, 29.0, 29.0, 29.0, 29.0, 29.0, 29.0, 29.0, 29.0, 29.0]}


In [ ]:
seed = 7
history, champion = evolve(seed)
print(history)

finish training
finish training
[8/10] max fitness: 13.0finish training
[10/10] max fitness: 16.1
{'expr_champion': [[1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.0*x_0 - x_1], [1.0*x_0 - x_1]], 'fitness_champion': [13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 16.1, 16.1]}


In [ ]:
seed = 8
history, champion = evolve(seed)
print(history)

finish training
finish training
[2/10] max fitness: 12.0finish training
[3/10] max fitness: 12.0finish training
finish training
[5/10] max fitness: 12.0finish training
[7/10] max fitness: 12.0finish training
[9/10] max fitness: 12.0finish training
finish training
[10/10] max fitness: 12.0
{'expr_champion': [[-x_0**2/x_1 + x_1], [0], [0], [0], [0], [0], [0], [0], [0], [0]], 'fitness_champion': [12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0]}


In [ ]:
seed = 9
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[6/10] max fitness: 18.0finish training
finish training
[7/10] max fitness: 18.0finish training
[9/10] max fitness: 18.0finish training
[10/10] max fitness: 18.0
{'expr_champion': [[x_0 - 2.0], [1/x_1], [x_0 - x_1 - 1.0], [x_0 - x_1 - 1.0], [x_0 - x_1 - 1.0], [x_0 - x_1 - 1.0], [x_0 - x_1 - 1.0], [x_0 - 2.0], [x_0 - x_1 - 1.0], [x_0 - x_1 - 1.0]], 'fitness_champion': [18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0]}


In [ ]:
seed = 10
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[2/10] max fitness: 16.0finish training
[3/10] max fitness: 22.0finish training
[4/10] max fitness: 22.0finish training
finish training
[5/10] max fitness: 22.0finish training
[7/10] max fitness: 22.0finish training
[10/10] max fitness: 22.0
{'expr_champion': [[x_0*x_1 + x_1 - 1.0], [x_0 - 1.0], [x_1], [x_1], [x_1], [x_1], [x_1], [x_1], [x_1], [x_1]], 'fitness_champion': [16.0, 16.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0]}


In [ ]:
seed = 6
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[3/10] max fitness: 18.0finish training
[4/10] max fitness: 18.0finish training
[10/10] max fitness: 18.0
{'expr_champion': [[x_1*(x_1 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)], [x_1*(x_0 - 1)]], 'fitness_champion': [14.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0]}


In [ ]:
seed = 7
history, champion = evolve(seed)
print(history)

finish training
finish training
[8/10] max fitness: 15.0finish training
[10/10] max fitness: 15.0
{'expr_champion': [[1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000], [1.00000000000000]], 'fitness_champion': [15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0]}


In [ ]:
seed = 8
history, champion = evolve(seed)
print(history)

finish training
finish training
[2/10] max fitness: 14.0finish training
[3/10] max fitness: 14.0finish training
finish training
[4/10] max fitness: 14.0finish training
[5/10] max fitness: 14.0finish training
[7/10] max fitness: 14.0finish training
[8/10] max fitness: 14.0

In [ ]:
seed = 9
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[6/10] max fitness: 16.0finish training
finish training
[7/10] max fitness: 16.0finish training
[9/10] max fitness: 18.0finish training
[10/10] max fitness: 18.0
{'expr_champion': [[x_0 - 2.0], [1/x_1], [x_0 - x_1 - 1.0], [x_0 - x_1 - 1.0], [x_0 - x_1 - 1.0], [x_0 - x_1 - 1.0], [x_0 - x_1 - 1.0], [1.00000000000000], [1.00000000000000], [1.00000000000000]], 'fitness_champion': [16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 18.0, 18.0, 18.0]}


In [ ]:
seed = 10
history, champion = evolve(seed)
print(history)

finish training
finish training
finish training
[2/10] max fitness: 17.0finish training
[3/10] max fitness: 17.0finish training
[4/10] max fitness: 17.0finish training
finish training
[7/10] max fitness: 17.0finish training
[9/10] max fitness: 17.0finish training
[10/10] max fitness: 17.0
{'expr_champion': [[x_0*x_1 + x_1 - 1.0], [x_0 - 1.0], [x_0*x_1 + x_1 - 1.0], [x_0 - 1.0], [(x_1 - 1.0)/(x_0 - x_1)], [x_0 - 1.0], [(x_1 - 1.0)/(x_0 - x_1)], [(x_1 - 1.0)/(x_0 - x_1)], [(x_1 - 1.0)/(x_0 - x_1)], [(x_1 - 1.0)/(x_0 - x_1)]], 'fitness_champion': [17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0, 17.0]}


## =================================================

In [ ]:
fitness = 0
nexp = 3
for exp in range(nexp): 
  torch.manual_seed(exp)
  data = torch.rand(M, T, N).le(F / 1000).float()
  targets = torch.rand(M).gt(0.5).int()
  traces = []
  # ugly
  for i in range(data.shape[1]):
      if i == 0:
          traces.append(data[:, i, :])
      else:
          trace = traces[-1] * O_DECAY + data[:, i, :]  #*(1 - O_DECAY)
          traces.append(trace)
  traces = torch.stack(traces, dim=1)
  param = {}
  # weights from N(0, 1000) like in paper, sparsity of 0.2 (20% of connections zero)
  param["W"] = torch.randn(1, N) * 7 * (torch.rand(1, N) < 0.8).float()
  #print(param)
  acc = []
  fit = []
  ne = 200
  nf = 10
  for e in range(ne):
      correct = 0
      ytab = []
      for i in range(M):#(M):
          s = None
          y = 0
          E = None  # trace from paper
          for t in range(T): #T
              if E is None:
                  E = torch.zeros_like(param["W"])
              z, s = forward(data[i, t], s, param)
              y += z
              xi = traces[i, t] * z + s[3] * data[i, t] 
              E = E * E_DECAY + xi  #E = E * E_DECAY + (traces[i, t] * s[3]）  # no (1 - E_DECAY)
              reward = get_reward(z, targets[i])
              param = learn(param, E, reward)
          ytab.append(y.item())
          
      predictions = torch.Tensor([1 if n > sum(ytab)/M else 0 for n in ytab])
      correct = int(M-sum(abs(predictions-targets)).item())
      acc.append(correct/M)
      #print(f"epoch {e}: {correct}/{M}")
      if e >= (ne - nf):
        fit.append(correct)
  fitness += sum(fit)/nf        
fitness = fitness/nexp
print(fitness)

19.599999999999998















r54## GOOD RESULTS

## Test =====================================....................